In [35]:
import adcUtil  as adc
import math
def getStick():   
    xpos = adc.readADC(channel=0)
    ypos = adc.readADC(channel=1)
    return xpos, ypos

def getStickKey():
    xpos, ypos = getStick()
    xpos = (xpos -.5)*2
    if xpos == 0:
        xpos ==.001
    ypos = (ypos -.5)*2
    r = math.sqrt(xpos**2 + ypos**2)
    theta = math.atan(ypos/xpos)
    print("X,Y: ({0:.2f},{0:.2f})".format(xpos,ypos))
    return r, theta


In [ ]:
import sys
# !{sys.executable} -m pip install pynput
import numpy as np
import time
from pynput.keyboard import Key, Controller
import spidev 

import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BCM)

bblue = 23
byel = 24
bgreen = 25
bred = 26

spi = spidev.SpiDev()
# (0,0 or 1) depending on adc pin, 0 in this case
spi.open(0, 0)

GPIO.setup(bblue, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(byel, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(bgreen, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(bred, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)

keyboard = Controller()

while True:
    if GPIO.input(bblue):    
        keyboard.press(Key.down)
        time.sleep(.1)
        keyboard.release(Key.down)    
        print("blue")
    if GPIO.input(byel):    
        keyboard.press(Key.right)
        time.sleep(.1)
        keyboard.release(Key.right)
    if GPIO.input(bgreen):    
        keyboard.press(Key.up)
        time.sleep(.1)
        keyboard.release(Key.up)
    if GPIO.input(bred):    
        keyboard.press(Key.left)
        time.sleep(.1)
        keyboard.release(Key.left)
    xpos, ypos = getStick()
    

    
# from pynput.keyboard import Key, Controller


In [43]:
import time
import adcUtil  as adc
import math

XMAX =1.0806451612903225
YMAX =1.0838709677419354
YMIN = 0
XMIN = 0

to = time.time();

while time.time()<(to+10) or False:
    #r, theta = getStickKey()
    x_pos, y_pos = getStick()
    print("Points: ({0:.2f},{0:.2f})".format(x_pos, y_pos))
    time.sleep(.4)
    
#r, theta = getStickKey()
print("Polar: ({0:.2f},{0:.2f})".format(r, theta))

Points: (0.53,0.53)
Points: (0.52,0.52)
Points: (0.51,0.51)
Points: (0.52,0.52)
Points: (0.53,0.53)
Points: (0.53,0.53)
Points: (0.53,0.53)
Points: (0.52,0.52)
Points: (0.54,0.54)
Points: (0.53,0.53)
Points: (0.53,0.53)
Points: (0.51,0.51)
Points: (0.53,0.53)
Points: (0.53,0.53)
Points: (0.52,0.52)
Points: (0.53,0.53)
Points: (0.53,0.53)
Points: (0.53,0.53)
Points: (0.53,0.53)
Points: (0.52,0.52)
Points: (0.43,0.43)
Points: (0.43,0.43)
Points: (0.52,0.52)
Points: (0.53,0.53)
Points: (0.53,0.53)
Polar: (0.17,0.17)
